## <b>4.3 PREPROCESSAMENT I ANÀLISI DE DADES</b>

### <b>4.3.4 Enginyeria de variables avançada</b>

#### <b>4.3.4.1 Objectiu i setup</b>

In [4]:
# ============================================================
# 4.3.4.1 ENGINYERIA DE VARIABLES AVANÇADA
# SETUP INICIAL I DEFINICIÓ DE DATASETS PER MODEL
# ============================================================
# Objectiu:
#   - Carregar el dataset final preprocessat (transformed_motor_insurance.csv).
#   - Definir llistes de variables per model:
#       * Model de freqüència (Has_claims_year).
#       * Model de severitat (Cost_claims_year).
#       * Model de ràtio econòmica (Claims_to_premium_ratio).
#   - Crear la variable Policy_year i la partició temporal train/test.
#   - Construir df_freq_base, df_sev_base i df_ratio_base com a bases de treball.
#   - Comprovar nuls i desar aquests datasets per als subapartats següents.
#
#   → Versió corregida:
#       * S’eviten columnes duplicades en les llistes de variables.
#       * Es comprova explícitament si hi ha columnes duplicades
#         als DataFrames resultants i, si n’hi ha, es deixen únicament
#         les primeres aparicions.
# ============================================================

import os
import numpy as np
import pandas as pd

# Opcions de display per veure millor taules amples
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

# ------------------------------------------------------------
# Funció auxiliar: assegurar que NO hi ha columnes duplicades
# ------------------------------------------------------------
def ensure_no_duplicate_columns(df_in: pd.DataFrame, name: str) -> pd.DataFrame:
    """
    Comprova si un DataFrame té columnes duplicades.
    - Si NO n’hi ha: només informa i retorna df tal qual.
    - Si n’hi ha:
        * imprimeix els noms duplicats,
        * es queda només amb la primera aparició de cada columna
          (df.loc[:, ~df.columns.duplicated()]),
        * informa de quantes columnes s’han eliminat.
    """
    dup_mask = df_in.columns.duplicated()
    if dup_mask.any():
        dup_cols = df_in.columns[dup_mask]
        print(f"\nATENCIÓ: S'han detectat columnes duplicades a {name}:")
        print(list(dup_cols))

        df_out = df_in.loc[:, ~dup_mask].copy()
        n_removed = df_in.shape[1] - df_out.shape[1]
        print(f"   → S'han eliminat {n_removed} columna/es duplicada/es de {name}.")
        print(f"   → Nombre final de columnes a {name}: {df_out.shape[1]}")
        return df_out
    else:
        print(f"\nSense columnes duplicades a {name}. Nombre de columnes: {df_in.shape[1]}")
        return df_in

# ------------------------------------------------------------
# 0) Càrrega del dataset final
# ------------------------------------------------------------

data_path = "transformed_motor_insurance.csv"

# Llegim el dataset principal de treball per als models.
# Aquest fitxer és el "canònic" després de totes les transformacions prèvies.
df = pd.read_csv(
    data_path,
    sep=",",        # separador de camp estàndard CSV
    encoding="utf-8"
)

print("Dataset carregat des de:", data_path)
print("Dimensions df original:", df.shape)

# ------------------------------------------------------------
# 1) Definició de variables clau per model
# ------------------------------------------------------------

# Noms de variables objectiu i identificador
target_freq = "Has_claims_year"          # target freqüència (0/1)
target_sev  = "Cost_claims_year"         # target severitat (cost anual)
ratio_col   = "Claims_to_premium_ratio"  # ràtio econòmica cost/prima
id_col      = "ID"                       # identificador únic de pòlissa/registre

# Llista base de variables explicatives per al model de FREQÜÈNCIA
freq_features_base = [
    "Driver_age", "Licence_age", "Vehicle_age",
    "Has_lapse", "Policy_duration",
    "Second_driver", "Area", "Type_risk", "Type_fuel",
    "Has_claims_history",
    "Value_vehicle", "Power", "Premium",
    "Seniority", "Policies_in_force",
    "Distribution_channel"
]

# Llista base de variables explicatives per al model de SEVERITAT
sev_features_base = [
    "Vehicle_age", "Value_vehicle", "Power",
    "Type_risk", "Area", "Policy_duration",
    "Claims_to_premium_ratio",   # també usada com a predictor de severitat
    "Weight", "Cylinder_capacity", "Premium", "Length"
]

# Variables de control i qualitat (no necessàriament entren al model com a predictors)
# IMPORTANT: aquí s’evita tornar a incloure Has_lapse, que ja és a freq_features_base,
#            per no generar duplicats innecessaris a freq_vars.
control_features = [
    "Licence_incoherent_flag",
    "Policy_incoherent_flag",
    "Lapse_incoherent_flag",
    "Length_missing_flag"
]

print("\nVariables base de freqüència:", len(freq_features_base))
print("Variables base de severitat :", len(sev_features_base))
print("Variables de control       :", len(control_features))

# ------------------------------------------------------------
# 2) Any de pòlissa i partició temporal train/test
# ------------------------------------------------------------

# Data de referència per definir l'any de pòlissa
date_col = "Date_last_renewal"
if date_col not in df.columns:
    raise ValueError(f"No existeix {date_col} al dataset. Revisa el fitxer d'entrada.")

# Convertim Date_last_renewal a datetime per poder extreure l'any
df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
df["Policy_year"] = df[date_col].dt.year

# Eliminem registres sense any vàlid (NaN a Policy_year)
df = df.dropna(subset=["Policy_year"]).copy()
df["Policy_year"] = df["Policy_year"].astype(int)

# Llista d'anys disponibles al dataset
years = sorted(df["Policy_year"].unique())
print("\nAnys disponibles a Policy_year:", years)

# Segons anàlisi temporal prèvia:
#   - train: anys anteriors a 2018
#   - test : 2018 i posteriors
train_years = [y for y in years if y < 2018]
test_years  = [y for y in years if y >= 2018]

# Etiqueta de conjunt (set_type) per a cada registre
df["set_type"] = np.where(df["Policy_year"].isin(train_years), "train", "test")

print("\nDistribució train/test:")
print(df["set_type"].value_counts())

print("\nDistribució per any i set_type:")
print(df.groupby(["Policy_year", "set_type"]).size())

# ------------------------------------------------------------
# 3) Datasets base per a cada model
# ------------------------------------------------------------

# ------------------------------
# A) FREQÜÈNCIA: df_freq_base
# ------------------------------
# Inclou:
#   - ID, Policy_year, set_type
#   - predictors de freq (freq_features_base)
#   - targets i ràtio (target_freq, target_sev, ratio_col)
#   - flags de control (control_features)
freq_vars = (
    [id_col, "Policy_year", "set_type"] +
    freq_features_base +
    [target_freq, target_sev, ratio_col] +
    control_features
)

# ❗ PAS IMPORTANT: eliminar duplicats mantenint l’ordre
#    (si accidentalment una variable aparegués en més d’una llista)
freq_vars = list(dict.fromkeys(freq_vars))

# Ens assegurem que només usem columnes que realment existeixen al df
freq_vars = [v for v in freq_vars if v in df.columns]

# DataFrame base de freqüència
df_freq_base = df[freq_vars].copy()
print("\ndf_freq_base creat (abans de revisar duplicats). Dimensions:", df_freq_base.shape)

# Comprovem i corrgim columnes duplicades (en principi no n'hi hauria d’haver)
df_freq_base = ensure_no_duplicate_columns(df_freq_base, "df_freq_base")

# ------------------------------
# B) SEVERITAT: df_sev_base
# ------------------------------
# Només sinistres amb cost > 0:
#   - Has_claims_year = 1
#   - Cost_claims_year != "0,0" (segons format original)
#   - Cost_claims_year no nul
mask_sev = (
    (df[target_freq] == 1) &        # només pòlisses amb almenys un sinistre
    (df[target_sev] != "0,0") &     # excloure sinistres amb cost zero escrit "0,0" (si fos text)
    (df[target_sev].notna())        # i sense valors nuls
)

df_sev = df.loc[mask_sev].copy()

sev_vars = (
    [id_col, "Policy_year", "set_type"] +   # identificador i any
    sev_features_base +                     # predictors de severitat
    [target_sev, target_freq, ratio_col] +  # target de cost + referència de freq i ràtio
    control_features                        # flags de qualitat
)

# Eliminar duplicats mantenint l'ordre (mateix criteri que abans)
sev_vars = list(dict.fromkeys(sev_vars))

# Ens quedem només amb les columnes que existeixen al subset df_sev
sev_vars = [v for v in sev_vars if v in df_sev.columns]

# DataFrame base de severitat
df_sev_base = df_sev[sev_vars].copy()
print("df_sev_base creat (abans de revisar duplicats). Dimensions:", df_sev_base.shape)

# Comprovem i corregim columnes duplicades
df_sev_base = ensure_no_duplicate_columns(df_sev_base, "df_sev_base")

# ------------------------------
# C) RÀTIO ECONÒMICA: df_ratio_base
# ------------------------------
# Model amb Claims_to_premium_ratio com a target principal.
ratio_vars = (
    [id_col, "Policy_year", "set_type"] +   # info d’identificador i temps
    freq_features_base +                    # predictors similars al model de freqüència
    [ratio_col, target_freq, target_sev] +  # target de ràtio + referències
    control_features                        # flags de control
)

# Eliminar duplicats mantenint l'ordre
ratio_vars = list(dict.fromkeys(ratio_vars))

ratio_vars = [v for v in ratio_vars if v in df.columns]

# DataFrame base per al model de ràtio econòmica
df_ratio_base = df[ratio_vars].copy()
print("df_ratio_base creat (abans de revisar duplicats). Dimensions:", df_ratio_base.shape)

# Comprovem i corregim columnes duplicades
df_ratio_base = ensure_no_duplicate_columns(df_ratio_base, "df_ratio_base")

# ------------------------------------------------------------
# 4) Comprovacions ràpides de nuls
# ------------------------------------------------------------

def quick_missing_report(df_in, name):
    """
    Mostra un petit resum dels valors nuls per DataFrame:
      - Llista variables amb almenys 1 nul.
      - Ordenades de més a menys nuls.
    """
    missing = df_in.isna().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    print(f"\n--- Valors nuls a {name} ---")
    if missing.empty:
        print("Sense nuls (excepte possibles nuls estructurals).")
    else:
        print(missing)

# Comprovem nuls als tres datasets base
quick_missing_report(df_freq_base,  "df_freq_base")
quick_missing_report(df_sev_base,   "df_sev_base")
quick_missing_report(df_ratio_base, "df_ratio_base")

# ------------------------------------------------------------
# 5) Desa datasets base
# ------------------------------------------------------------

# Creem carpeta per a dades processades si no existeix
os.makedirs("data/processed", exist_ok=True)

# Rutes de sortida
freq_path  = os.path.join("data/processed", "df_freq_base.csv")
sev_path   = os.path.join("data/processed", "df_sev_base.csv")
ratio_path = os.path.join("data/processed", "df_ratio_base.csv")

# Exportem a CSV (sense índex)
df_freq_base.to_csv(freq_path, index=False)
df_sev_base.to_csv(sev_path, index=False)
df_ratio_base.to_csv(ratio_path, index=False)

print(f"\ndf_freq_base desat a:  {freq_path}")
print(f"df_sev_base desat a:   {sev_path}")
print(f"df_ratio_base desat a: {ratio_path}")

print("\n4.3.4.1 complet - Setup llest per a transformacions avançades (sense columnes duplicades).")


Dataset carregat des de: transformed_motor_insurance.csv
Dimensions df original: (105555, 44)

Variables base de freqüència: 16
Variables base de severitat : 11
Variables de control       : 4

Anys disponibles a Policy_year: [2015, 2016, 2017, 2018]

Distribució train/test:
set_type
train    69740
test     35815
Name: count, dtype: int64

Distribució per any i set_type:
Policy_year  set_type
2015         train        4559
2016         train       31428
2017         train       33753
2018         test        35815
dtype: int64

df_freq_base creat (abans de revisar duplicats). Dimensions: (105555, 26)

Sense columnes duplicades a df_freq_base. Nombre de columnes: 26
df_sev_base creat (abans de revisar duplicats). Dimensions: (19646, 20)

Sense columnes duplicades a df_sev_base. Nombre de columnes: 20
df_ratio_base creat (abans de revisar duplicats). Dimensions: (105555, 26)

Sense columnes duplicades a df_ratio_base. Nombre de columnes: 26

--- Valors nuls a df_freq_base ---
Policy_durat

#### <b>4.3.4.2 Transformacions de variables numèriques</b>

In [6]:
# ============================================================
# 4.3.4.2 TRANSFORMACIONS DE VARIABLES NUMÈRIQUES
# ============================================================
# Aquest script té com a objectiu crear transformacions per a models
# actuarials i de Machine Learning en l'àmbit d’assegurança de motor.
# Les transformacions es construeixen a partir de 3 dataframes base que
# prèviament s'han creat després de tractar nuls i incongruències temporals.
# 
# Points clau del procés:
#  1) Assegurar que les variables són numèriques reals (float/int)
#  2) Crear transformacions logarítmiques per a variables altament asimètriques
#  3) Aplicar "capping" conservador amb percentils 1% i 99% (winsorització)
#  4) Normalitzar (estandarditzar) variables capejades per a models gradient
#  5) Binning per a models lineals/GLM per reforçar la relació lineal
#  6) Imputació simple final per garantir l'absència de NaN no estructurals
#  7) Guardar datasets transformats resultants
# 
# Nota CRUCIAL:
#  - NO fem str.replace(",", ".") perquè estem carregant el dataset principal
#    amb decimals en punt `.` (transformed_motor_insurance.csv)
#  - Els 3 CSV base es van guardar sense modificacions europees de decimals
# ============================================================

# Importem les llibreries necessàries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler   # Per estandarditzar variables numèriques
from sklearn.preprocessing import KBinsDiscretizer # Per discretitzar variables (binning en quantils)

# ------------------------------------------------------------
# 0) Carrega datasets base
# ------------------------------------------------------------

# Carreguem els 3 dataframes base (subsets de variables segons cada model)
df_freq = pd.read_csv("data/processed/df_freq_base.csv")   # Taula base per a model de freqüència
df_sev  = pd.read_csv("data/processed/df_sev_base.csv")    # Taula base per a model de severitat (només sinistres)
df_ratio = pd.read_csv("data/processed/df_ratio_base.csv") # Taula base per a model de ràtio econòmica (cost/prima)

# Imprimim dimensions de cada DF
print("Freq:", df_freq.shape, "| Sev:", df_sev.shape, "| Ratio:", df_ratio.shape)

# ------------------------------------------------------------
# 1) Assegurar que totes les variables clau són numèriques
# ------------------------------------------------------------

def ensure_numeric(df, cols, name="df"):
    """
    Converteix les columnes de `cols` a numèric (float o int segons pandas ho determini).
    Si algun valor no és convertible, el transforma a NaN.
    També imprimeix quants NaN nous s’han generat per conversió.
    """
    for col in cols:
        if col in df.columns:  # Només processem la columna si existeix al dataframe
            before_nulls = df[col].isna().sum()  # Comptem nuls abans de convertir
            df[col] = pd.to_numeric(df[col], errors="coerce")  # Converteix a numèric tolerant errors
            after_nulls  = df[col].isna().sum()  # Comptem nuls després de convertir
            new_nans = after_nulls - before_nulls  # Diferència = NaN creats en la coerció
            if new_nans > 0:
                # Avís si la conversió ha creat nous NaN
                print(f"{name} — La columna '{col}' ha generat {new_nans} NaN nous en convertir a numèrica.")
    return df

# Llista de columnes numèriques que s’han de garantir com a numèriques
numeric_cols = [
    "Driver_age",        # Edat del conductor (pot estar en float)
    "Licence_age",       # Antiguitat del carnet (float)
    "Vehicle_age",       # Antiguitat del vehicle (int o float)
    "Power",             # Potència del vehicle
    "Value_vehicle",     # Valor del vehicle assegurat
    "Premium",           # Prima anual
    "Cylinder_capacity", # Cilindrada
    "Weight",            # Pes del vehicle
    "Length",            # Longitud del vehicle
    "Cost_claims_year",  # Cost dels sinistres anuals
    "Claims_to_premium_ratio" # Ràtio cost/prima
]

# Apliquem el cast/endure numeric als 3 dataframes
df_freq  = ensure_numeric(df_freq,  numeric_cols, name="df_freq")
df_sev   = ensure_numeric(df_sev,   numeric_cols, name="df_sev")
df_ratio = ensure_numeric(df_ratio, numeric_cols, name="df_ratio")

print("\nCast a float/int aplicat quan calia.")

# ------------------------------------------------------------
# 2) Transformacions logarítmiques (log1p)
# ------------------------------------------------------------

# Variables que típicament tenen cues llargues i es beneficien d'un log per estabilitzar variància
log_vars = ["Cost_claims_year", "Value_vehicle", "Power"]

for col in log_vars:
    if col in df_freq.columns:
        # log1p = log(1 + x) per evitar errors amb 0 o valors molt petits
        # La nova columna porta el sufix "_log"
        df_freq[col+"_log"] = np.log1p(df_freq[col])
    if col in df_sev.columns:
        df_sev[col+"_log"] = np.log1p(df_sev[col])

print("Transformacions log1p aplicades:", log_vars)

# ------------------------------------------------------------
# 3) Winsorització / Capping
# ------------------------------------------------------------

def winsorize_series(s, lower=0.01, upper=0.99):
    """
    Aplica 'capping' basat en percentils 1% i 99%.
    No elimina registres: només limita valors a les cues extremes.
    """
    lo = s.quantile(lower)  # Percentil inferior (1%)
    hi = s.quantile(upper)  # Percentil superior (99%)
    return np.clip(s, lo, hi)  # Retorna valors tallats entre [lo, hi]

# Columnes que winsoritzarem (versió capejada)
winsor_vars = ["Value_vehicle", "Power", "Premium", "Cost_claims_year"]

for col in winsor_vars:
    if col in df_freq.columns:
        # Generem la versió capejada "_cap"
        df_freq[col+"_cap"] = winsorize_series(df_freq[col])
    if col in df_sev.columns:
        df_sev[col+"_cap"] = winsorize_series(df_sev[col])

print("Capping aplicat a:", winsor_vars)

# ------------------------------------------------------------
# 4) Escalat per a models basats en gradient (StandardScaler)
# ------------------------------------------------------------

scaler = StandardScaler()

# Columnes que estandarditzarem (només si existeixen i després d'haver-les capejat)
scale_vars = ["Value_vehicle_cap", "Power_cap", "Premium_cap"]

for col in scale_vars:
    if col in df_freq.columns:
        # Fit + transform (s'espera que el DF_freq sigui prou gran per estabilitzar scale)
        df_freq[col+"_scaled"] = scaler.fit_transform(df_freq[[col]])
    if col in df_sev.columns:
        df_sev[col+"_scaled"] = scaler.fit_transform(df_sev[[col]])

print("Escalat (z-score) aplicat quan calia.")

# ------------------------------------------------------------
# 5) Binning per models lineals / GLM (KBinsDiscretizer)
# ------------------------------------------------------------

# Variables que discretitzarem en 6 trams (quantils) buscant linealitat per GLM
bin_vars = ["Driver_age", "Vehicle_age", "Power", "Value_vehicle"]

for col in bin_vars:
    if col in df_freq.columns:
        # Discretitzem based en quantils, codificat com ordinal
        kb = KBinsDiscretizer(
            n_bins=6,
            encode='ordinal',
            strategy='quantile',
            quantile_method='linear'  # mètode estable per evitar warnings futurs
        )
        df_freq[col + "_bin"] = kb.fit_transform(df_freq[[col]])

    if col in df_sev.columns:
        kb = KBinsDiscretizer(
            n_bins=6,
            encode='ordinal',
            strategy='quantile',
            quantile_method='linear'
        )
        df_sev[col + "_bin"] = kb.fit_transform(df_sev[[col]])

print("Binning aplicat a:", bin_vars)

# ------------------------------------------------------------
# 6) Imputació final simple (NA → -1)
# ------------------------------------------------------------

# Substituïm NaN residuals amb -1 per evitar errors als models ML.
# És conservador: no introdueix informació nova, només evita fallades de conversió.
df_freq  = df_freq.fillna(-1)
df_sev   = df_sev.fillna(-1)
df_ratio = df_ratio.fillna(-1)

print("Imputació simple final completada (valors NA → -1).")

# ------------------------------------------------------------
# 7) Guardem datasets transformats
# ------------------------------------------------------------

df_freq.to_csv("data/processed/df_freq_num_transformed.csv", index=False, encoding="utf-8")
df_sev.to_csv("data/processed/df_sev_num_transformed.csv", index=False, encoding="utf-8")
df_ratio.to_csv("data/processed/df_ratio_num_transformed.csv", index=False, encoding="utf-8")

print("Datasets transformats desats a data/processed/")


Freq: (105555, 26) | Sev: (19646, 20) | Ratio: (105555, 26)

Cast a float/int aplicat quan calia.
Transformacions log1p aplicades: ['Cost_claims_year', 'Value_vehicle', 'Power']
Capping aplicat a: ['Value_vehicle', 'Power', 'Premium', 'Cost_claims_year']
Escalat (z-score) aplicat quan calia.
Binning aplicat a: ['Driver_age', 'Vehicle_age', 'Power', 'Value_vehicle']
Imputació simple final completada (valors NA → -1).
Datasets transformats desats a data/processed/


#### <b>4.3.4.3 Transformació i codificació de variables categòriques</b>

In [8]:
# ============================================================
# 4.3.4.3 TRANSFORMACIÓ I CODIFICACIÓ DE VARIABLES CATEGÒRIQUES
# ============================================================
# Objectiu:
#   - Preparar les variables categòriques per als diferents models.
#   - Aplicar codificació nominal (One-Hot Encoding) per a GLM i GAM.
#   - Aplicar Target Encoding per a models basats en gradient (GBM).
#   - Realitzar codificació ordinal per a variables amb ordre inherent
#     (p. ex. Type_risk).
#   - Agrupar categories rares per garantir estabilitat (p. ex. Type_fuel="OTHER").
#   - Incorporar informació temporal (freqüències per any) per captar drift.
#   - Generar datasets separats per a GLM/GAM i Gradient Boosting.
# ============================================================

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

# ------------------------------------------------------------
# 0) Càrrega dels datasets numèrics transformats (4.3.4.2)
# ------------------------------------------------------------
# Aquests fitxers ja inclouen:
#   - variables numèriques netes i transformades
#   - splits train/test
#   - imputació de NaN amb valors sentinel (-1) quan cal

df_freq = pd.read_csv("data/processed/df_freq_num_transformed.csv")

print("Data carregada:")
print("Freq:", df_freq.shape, "| Sev:", df_sev.shape, "| Ratio:", df_ratio.shape)

# ------------------------------------------------------------
# 1) Llistes de variables categòriques
# ------------------------------------------------------------
# Definim les columnes categòriques que volem codificar.
# Elles són compartides, principalment, en el model de freqüència.

cat_vars = [
    "Type_risk",            # Tipus de risc (1, 2, 3, 4)
    "Area",                 # Zona geogràfica
    "Type_fuel",            # Tipus de combustible
    "Distribution_channel", # Canal de distribució
    "Second_driver",        # Indicador segon conductor (0/1)
    "Has_lapse",            # Indicador de cancel·lació
    "Has_claims_history"    # Històric de sinistres (0/1)
]

# ------------------------------------------------------------
# 2) Agrupació de categories rares
# ------------------------------------------------------------
# Funció per agrupar categories amb pes molt baix (< threshold)
# en una categoria comuna "OTHER", per millorar estabilitat en models.

def rare_category_grouping(df, col, threshold=0.01):
    """
    Agrupa categories rares (freqüència relativa < threshold)
    en una categoria comuna 'OTHER'.
    """
    freq = df[col].value_counts(normalize=True)  # distribució relativa per categoria
    rare_cats = freq[freq < threshold].index     # categories per sota del llindar

    if len(rare_cats) > 0:
        df[col] = df[col].replace(rare_cats, "OTHER")  # substituïm categories rares per "OTHER"
    return df

# Apliquem l’agrupació a Type_fuel (tant a freq, com a sev i ratio)
for col in ["Type_fuel"]:
    # Freqüència
    if col in df_freq.columns:
        df_freq = rare_category_grouping(df_freq, col)

    # Severitat
    if col in df_sev.columns:
        df_sev = rare_category_grouping(df_sev, col)

    # Ràtio
    if col in df_ratio.columns:
        df_ratio = rare_category_grouping(df_ratio, col)

print("Categories rares agrupades (si escau) a Type_fuel.")

# ------------------------------------------------------------
# 3) Codificació ordinal (Type_risk)
# ------------------------------------------------------------
# Suposem que Type_risk té un ordre natural: 1 < 2 < 3 < 4.
# La codificació ordinal preserva aquesta ordre en una variable numèrica.

if "Type_risk" in df_freq.columns:
    # Definim l'ordre explícit de les categories
    ord_enc = OrdinalEncoder(categories=[['1', '2', '3', '4']])

    # Fit + transform sobre df_freq (dataset principal)
    df_freq["Type_risk_ord"] = ord_enc.fit_transform(df_freq[["Type_risk"]])
    # Transform coherent sobre la resta de datasets
    df_sev["Type_risk_ord"] = ord_enc.transform(df_sev[["Type_risk"]])
    df_ratio["Type_risk_ord"] = ord_enc.transform(df_ratio[["Type_risk"]])

print("Codificació ordinal aplicada a Type_risk.")

# ------------------------------------------------------------
# 4) One-Hot Encoding per GLM/GAM
# ------------------------------------------------------------
# Per a models lineals (GLM/GAM) és habitual usar codificació one-hot,
# que converteix cada categoria en una dummy (0/1) per evitar ordres artificials.

ohe = OneHotEncoder(handle_unknown="ignore")  # ignorem categories desconegudes en predicció

# Copiem df_freq com a base per dataset GLM
df_glm = df_freq.copy()

# Ajustem el OneHotEncoder sobre les variables categòriques
glm_encoded = ohe.fit_transform(df_glm[cat_vars]).toarray()

# Obtenim els noms de les noves columnes OHE (una per categoria)
ohe_cols = ohe.get_feature_names_out(cat_vars)

# Afegim aquestes columnes al DataFrame GLM
df_glm[ohe_cols] = glm_encoded

# Eliminem les columnes categòriques originals (ara codificades en dummies)
df_glm = df_glm.drop(columns=cat_vars)

print("One-Hot Encoding aplicat (dataset GLM/GAM).")
print("Dimensions df_glm:", df_glm.shape)

# ------------------------------------------------------------
# 5) Target Encoding per models basats en gradient
# ------------------------------------------------------------
# Per models tipus Gradient Boosting (XGBoost, LightGBM, CatBoost…),
# el Target Encoding és una manera compacta de codificar categories:
# substitueix cada categoria pel valor mitjà del target dins d’aquesta categoria.
#
# Aquí ho apliquem sobre el target de freqüència: Has_claims_year.

def target_encode(df, col, target):
    """
    Codifica una variable categòrica `col` substituint cada categoria
    per la mitjana del `target` (target encoding simple).
    """
    means = df.groupby(col)[target].mean()  # mitjana del target per categoria
    return df[col].map(means)               # mapegem cada fila al valor mitjà corresponent

# Base per dataset GBM (freqüència)
df_gbm = df_freq.copy()

# Apliquem target encoding a cada variable categòrica
for col in cat_vars:
    df_gbm[col + "_te"] = target_encode(df_gbm, col, "Has_claims_year")

# Eliminem les categòriques originals (treballarem amb les versions _te)
df_gbm = df_gbm.drop(columns=cat_vars)

print("Target Encoding aplicat (dataset Gradient Boosting).")
print("Dimensions df_gbm:", df_gbm.shape)

# ------------------------------------------------------------
# 6) Guardar datasets finals
# ------------------------------------------------------------
# Guardem:
#   - df_glm: dataset per GLM/GAM amb OHE
#   - df_gbm: dataset per models de gradient amb Target Encoding
#   - df_glm_temp / df_gbm_temp: versions amb agregats temporals per drift

df_glm.to_csv("data/processed/df_freq_cat_glm.csv", index=False)
df_gbm.to_csv("data/processed/df_freq_cat_gbm.csv", index=False)

print("Datasets categòrics transformats i desats correctament.")


Data carregada:
Freq: (105555, 40) | Sev: (19646, 33) | Ratio: (105555, 26)
Categories rares agrupades (si escau) a Type_fuel.
Codificació ordinal aplicada a Type_risk.
One-Hot Encoding aplicat (dataset GLM/GAM).
Dimensions df_glm: (105555, 51)
Target Encoding aplicat (dataset Gradient Boosting).
Dimensions df_gbm: (105555, 41)
Datasets categòrics transformats i desats correctament.


#### <b>4.3.4.4	Variables derivades i indicadors compostos</b>

In [10]:
# ============================================================
# 4.3.4.4 VARIABLES DERIVADES I INDICADORS COMPOSTOS
# ============================================================
# Objectiu:
#   - Construir variables combinades de risc (driver × vehicle).
#   - Crear indicadors de comportament: lapse, recency, historial.
#   - Generar ràtios econòmics derivats del valor, potència i prima.
#   - Afegir interaccions seleccionades a partir de l’EDA.
#   - Evitar explosió dimensional: només interaccions d’alt valor.
#   - Guardar datasets per a la partició i modelatge de 4.3.5.
#
# Nota:
#   - Es parteix de datasets ja transformats numèricament i categòricament:
#       * df_freq_cat_glm.csv  → per GLM/GAM (OHE)
#       * df_freq_cat_gbm.csv  → per GBM (target encoding)
#       * df_sev_num_transformed.csv → severitat numèrica transformada
# ============================================================

# Opcions de display per inspecció en consola
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

# ------------------------------------------------------------
# 0) Càrrega de datasets transformats previs
# ------------------------------------------------------------

freq_glm_path = "data/processed/df_freq_cat_glm.csv"          # Freqüència per GLM/GAM
freq_gbm_path = "data/processed/df_freq_cat_gbm.csv"          # Freqüència per GBM (target encoding)
sev_num_path  = "data/processed/df_sev_num_transformed.csv"   # Severitat numèrica

df_glm = pd.read_csv(freq_glm_path)
df_gbm = pd.read_csv(freq_gbm_path)
df_sev = pd.read_csv(sev_num_path)

print("Freq GLM:", df_glm.shape)
print("Freq GBM:", df_gbm.shape)
print("Severitat:", df_sev.shape)

# ------------------------------------------------------------
# 1) Helpers per crear interaccions i ràtios de forma segura
# ------------------------------------------------------------

def add_interaction(df, col_a, col_b, new_name=None):
    """
    Afegeix una nova columna d’interacció = df[col_a] * df[col_b],
    si ambdues columnes existeixen al DataFrame.
    """
    if col_a in df.columns and col_b in df.columns:
        # Nom per defecte si no se'n passa cap
        if new_name is None:
            new_name = f"{col_a}_x_{col_b}"
        df[new_name] = df[col_a] * df[col_b]
        print(f"   ➜ Creat: {new_name}")
    else:
        # Si manca alguna columna, informem i no fem res
        missing = [c for c in [col_a, col_b] if c not in df.columns]
        print(f"   ⚠ No s'ha creat interacció {col_a}×{col_b} (manca: {missing})")
    return df

def safe_ratio(df, num_col, den_col, new_name):
    """
    Crea un ràtio num_col / den_col si les columnes existeixen.
    Afegim un epsilon petit (1e-6) al denominador per evitar divisió per 0.
    """
    if num_col in df.columns and den_col in df.columns:
        df[new_name] = df[num_col] / (df[den_col] + 1e-6)
        print(f"   ➜ Creat ràtio: {new_name}")
    else:
        missing = [c for c in [num_col, den_col] if c not in df.columns]
        print(f"   No s'ha creat ràtio {new_name} (manca: {missing})")
    return df

# ------------------------------------------------------------
# 2) Interaccions numèriques de risc combinat (freqüència)
# ------------------------------------------------------------
# Interaccions i ràtios pensades per capturar relacions no lineals
# entre característiques del conductor, vehicle i economia.

print("\n[Freqüència - GLM] Interaccions numèriques principals:")

# Interacció edat conductor × potència
df_glm = add_interaction(df_glm, "Driver_age", "Power", "Driver_age_x_Power")

# Interacció antiguitat vehicle × valor vehicle
df_glm = add_interaction(df_glm, "Vehicle_age", "Value_vehicle", "Vehicle_age_x_Value_vehicle")

# Ràtio valor/potència (proxy de “valor per unitat de potència”)
df_glm = safe_ratio(df_glm, "Value_vehicle", "Power", "Value_to_power")

# Ràtio prima/valor (intensitat de prima respecte valor assegurat)
df_glm = safe_ratio(df_glm, "Premium", "Value_vehicle", "Premium_to_value")

print("\n[Freqüència - GBM] Interaccions numèriques principals:")

df_gbm = add_interaction(df_gbm, "Driver_age", "Power", "Driver_age_x_Power")
df_gbm = add_interaction(df_gbm, "Vehicle_age", "Value_vehicle", "Vehicle_age_x_Value_vehicle")

df_gbm = safe_ratio(df_gbm, "Value_vehicle", "Power", "Value_to_power")
df_gbm = safe_ratio(df_gbm, "Premium", "Value_vehicle", "Premium_to_value")

# ------------------------------------------------------------
# 3) Interaccions amb variables categòriques codificades
#    - GLM: Type_risk_ord × dummies d’Area
#    - GLM: Second_driver dummificada × Power
#    - GBM: interaccions entre columnes target-encoded
# ------------------------------------------------------------

print("\n[Freqüència - GLM] Interaccions Type_risk × Area:")

# Per GLM: usem Type_risk_ord (ordinal) i les dummies d’Area (Area_*)
if "Type_risk_ord" in df_glm.columns:
    # Busquem totes les columnes dummies d’Area (creades per l’OHE)
    area_cols = [c for c in df_glm.columns if c.startswith("Area_")]
    for ac in area_cols:
        new_name = f"Type_risk_ord_x_{ac}"
        df_glm[new_name] = df_glm["Type_risk_ord"] * df_glm[ac]
        print(f"   ➜ Creat: {new_name}")
else:
    print("   No hi ha Type_risk_ord a df_glm.")

print("\n[Freqüència - GLM] Interaccions Second_driver × Power:")

# Interacció entre les dummies de Second_driver i la potència
power_col = "Power"
second_driver_cols = [c for c in df_glm.columns if c.startswith("Second_driver_")]
for sd in second_driver_cols:
    new_name = f"{sd}_x_{power_col}"
    if power_col in df_glm.columns:
        df_glm[new_name] = df_glm[sd] * df_glm[power_col]
        print(f"   ➜ Creat: {new_name}")
    else:
        print(f"   ⚠ No s'ha creat {new_name} (manca Power).")

print("\n[Freqüència - GBM] Interaccions categòriques resumides:")

# Per GBM: treballem amb versions target-encoded (col_te)
# Ex.: Type_risk_te × Area_te
if "Type_risk_te" in df_gbm.columns and "Area_te" in df_gbm.columns:
    df_gbm["Type_risk_te_x_Area_te"] = df_gbm["Type_risk_te"] * df_gbm["Area_te"]
    print("   ➜ Creat: Type_risk_te_x_Area_te")

# Ex.: Second_driver_te × Power
if "Second_driver_te" in df_gbm.columns and "Power" in df_gbm.columns:
    df_gbm["Second_driver_te_x_Power"] = df_gbm["Second_driver_te"] * df_gbm["Power"]
    print("   ➜ Creat: Second_driver_te_x_Power")

# ------------------------------------------------------------
# 4) Variables derivades per severitat
#    (treballem sobre df_sev_num_transformed, sense OHE)
# ------------------------------------------------------------

print("\n[Severitat] Variables derivades i interaccions:")

# Interacció antiguitat vehicle × valor vehicle
df_sev = add_interaction(df_sev, "Vehicle_age", "Value_vehicle", "Vehicle_age_x_Value_vehicle")

# Interacció potència × valor vehicle
df_sev = add_interaction(df_sev, "Power", "Value_vehicle", "Power_x_Value_vehicle")

# Ràtio cost/prima (intensitat del cost respecte la prima)
df_sev = safe_ratio(df_sev, "Cost_claims_year", "Premium", "Sev_cost_to_premium")

# Ràtio valor/prima (relació entre capital cobert i prima)
df_sev = safe_ratio(df_sev, "Value_vehicle", "Premium", "Sev_value_to_premium")

# Interacció simple Type_risk × Area (si existeixen)
if "Type_risk" in df_sev.columns and "Area" in df_sev.columns:
    try:
        # Intentem assegurar que Type_risk sigui numèrica
        df_sev["Type_risk"] = pd.to_numeric(df_sev["Type_risk"], errors="coerce")
        df_sev["Type_risk_x_Area"] = df_sev["Type_risk"] * df_sev["Area"]
        print("   ➜ Creat: Type_risk_x_Area")
    except Exception as e:
        print("No s'ha pogut crear Type_risk_x_Area:", e)

# ------------------------------------------------------------
# 5) Desa datasets enriquits
# ------------------------------------------------------------

os.makedirs("data/processed", exist_ok=True)

freq_glm_fe_path = "data/processed/df_freq_fe_glm.csv"  # freq GLM amb feature engineering
freq_gbm_fe_path = "data/processed/df_freq_fe_gbm.csv"  # freq GBM amb feature engineering
sev_fe_path      = "data/processed/df_sev_fe.csv"       # severitat amb feature engineering

df_glm.to_csv(freq_glm_fe_path, index=False)
df_gbm.to_csv(freq_gbm_fe_path, index=False)
df_sev.to_csv(sev_fe_path, index=False)

print("\nDatasets enriquits desats:")
print("   -", freq_glm_fe_path)
print("   -", freq_gbm_fe_path)
print("   -", sev_fe_path)

print("\n4.3.4.4 complet - Variables derivades i interaccions creades correctament.")


Freq GLM: (105555, 51)
Freq GBM: (105555, 41)
Severitat: (19646, 33)

[Freqüència - GLM] Interaccions numèriques principals:
   ➜ Creat: Driver_age_x_Power
   ➜ Creat: Vehicle_age_x_Value_vehicle
   ➜ Creat ràtio: Value_to_power
   ➜ Creat ràtio: Premium_to_value

[Freqüència - GBM] Interaccions numèriques principals:
   ➜ Creat: Driver_age_x_Power
   ➜ Creat: Vehicle_age_x_Value_vehicle
   ➜ Creat ràtio: Value_to_power
   ➜ Creat ràtio: Premium_to_value

[Freqüència - GLM] Interaccions Type_risk × Area:
   ➜ Creat: Type_risk_ord_x_Area_0
   ➜ Creat: Type_risk_ord_x_Area_1

[Freqüència - GLM] Interaccions Second_driver × Power:
   ➜ Creat: Second_driver_0_x_Power
   ➜ Creat: Second_driver_1_x_Power

[Freqüència - GBM] Interaccions categòriques resumides:
   ➜ Creat: Type_risk_te_x_Area_te
   ➜ Creat: Second_driver_te_x_Power

[Severitat] Variables derivades i interaccions:
   ➜ Creat: Vehicle_age_x_Value_vehicle
   ➜ Creat: Power_x_Value_vehicle
   ➜ Creat ràtio: Sev_cost_to_premium
  

#### <b>4.3.4.5 Reducció de dimensionalitat i anàlisi multivariable</b>

In [12]:
# ============================================================
# 4.3.4.5 REDUCCIÓ DE DIMENSIONALITAT I ANÀLISI MULTIVARIANT
# ============================================================
# Objectiu:
#   - Avaluar la multicol·linearitat entre predictors (VIF, correlacions).
#   - Identificar variables redundants i candidates a eliminació per al GLM.
#   - Aplicar un PCA exploratori sobre el dataset GBM (només diagnòstic).
#   - Avaluar importàncies de variables amb un Gradient Boosting (freqüència).
#   - Generar una llista de variables recomanades a eliminar en GLM.
# ============================================================

import pandas as pd
import numpy as np
import warnings
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor

# ------------------------------------------------------------
# 0) Carregar datasets enriquits
# ------------------------------------------------------------
# Es carreguen els datasets resultants de l’enginyeria de variables:
#   - df_freq_fe_glm: model de freqüència per GLM, amb OHE + interaccions
#   - df_freq_fe_gbm: model de freqüència per GBM, amb target encoding + interaccions
#   - df_sev_fe: model de severitat, no es fa servir directament aquí però es carrega per coherència

df_glm = pd.read_csv("data/processed/df_freq_fe_glm.csv")
df_gbm = pd.read_csv("data/processed/df_freq_fe_gbm.csv")

print("Carregats:")
print("GLM:", df_glm.shape)
print("GBM:", df_gbm.shape)
print("SEV:", df_sev.shape)

# ------------------------------------------------------------
# 1) Seleccionar només variables numèriques
# ------------------------------------------------------------
# Per calcular VIF, correlacions i PCA necessitem només columnes numèriques.

def get_numeric(df):
    """Retorna només les columnes numèriques (int/float) del DataFrame."""
    return df.select_dtypes(include=[np.number])

num_glm = get_numeric(df_glm)

# ------------------------------------------------------------
# 1.1 Eliminar columnes constants o duplicades (abans del VIF)
# ------------------------------------------------------------
# Les columnes constants (std=0) o duplicades poden donar problemes al VIF
# (divide by zero, inverses de matrius singulars, etc.).
# Per això les eliminem prèviament del subconjunt per a GLM.

# Columnes constants: tenen <=1 valor diferent
constant_cols = [c for c in num_glm.columns if num_glm[c].nunique() <= 1]

# Columnes duplicades: vectors idèntics
duplicated_cols = num_glm.T[num_glm.T.duplicated()].index.tolist()

# Unim totes les columnes a eliminar
cols_to_remove = set(constant_cols + duplicated_cols)

if len(cols_to_remove) > 0:
    print("Columnes eliminades abans del VIF (constants/duplicades):")
    print(cols_to_remove)

# DataFrame numèric net per càlcul de VIF i correlacions
num_glm_clean = num_glm.drop(columns=list(cols_to_remove), errors="ignore")

# ------------------------------------------------------------
# 2) Càlcul de VIF amb control d’errors
# ------------------------------------------------------------
# El VIF (Variance Inflation Factor) mesura multicol·linearitat:
#   VIF_j = 1 / (1 - R^2_j)   on R^2_j és el R^2 de regressar la variable j
#   contra la resta.
# VIF > 10 (aprox.) s’acostuma a considerar alta multicol·linearitat.

def compute_vif(df):
    """
    Calcula el VIF per a totes les variables numèriques de df,
    excloent explícitament la variable objectiu (Has_claims_year) si existeix.
    """
    vif_data = []

    # Eliminem la variable objectiu del conjunt X per evitar VIF sobre ella
    X = df.drop(columns=["Has_claims_year"], errors="ignore").copy()

    # Eliminem de nou columnes amb desviació estàndard 0 per evitar divisions per zero
    const_cols = X.columns[X.std() == 0]
    if len(const_cols) > 0:
        X = X.drop(columns=const_cols)

    # Calculem el VIF per a cada columna restant
    for i in range(X.shape[1]):
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=RuntimeWarning)
            vif = variance_inflation_factor(X.values, i)
        vif_data.append((X.columns[i], vif))

    # Convertim a DataFrame per facilitar export i inspecció
    vif_df = pd.DataFrame(vif_data, columns=["Variable", "VIF"])
    # Substituïm infinits per NaN per evitar problemes posteriors
    vif_df["VIF"] = vif_df["VIF"].replace([np.inf, -np.inf], np.nan)
    return vif_df

# Càlcul de VIF sobre les variables numèriques netes del GLM
vif_glm = compute_vif(num_glm_clean)
vif_glm.to_csv("data/processed/vif_glm.csv", index=False)

print("VIF calculat per GLM i guardat a data/processed/vif_glm.csv.")

# ------------------------------------------------------------
# 3) Correlacions altes > |0.75|
# ------------------------------------------------------------
# Un cop netejat num_glm_clean, calculem la matriu de correlació absoluta.
# Identificarem parelles de variables amb |correlació| > 0.75 com a candidates
# a ser redundants (una de les dues pot ser eliminada al GLM).

corr_glm = num_glm_clean.corr().abs()

# Ens quedem només amb la part superior de la matriu (k=1 evita diagonal)
high_corr_pairs = (
    corr_glm.where(np.triu(np.ones(corr_glm.shape), k=1).astype(bool))
    .stack()
    .reset_index()
)
high_corr_pairs.columns = ["Var1", "Var2", "Correlation"]

# Filtre per correlació alta
high_corr_pairs = high_corr_pairs[high_corr_pairs["Correlation"] > 0.75]

# Guardem per anàlisi detallada posterior
high_corr_pairs.to_csv("data/processed/high_corr_pairs_glm.csv", index=False)
print("Parelles amb correlació alta identificades i desades a high_corr_pairs_glm.csv.")

# ------------------------------------------------------------
# 4) PCA exploratori sobre GBM
# ------------------------------------------------------------
# El PCA es fa només amb finalitats exploratòries: per veure quanta variància
# s’explica amb els primers components. No es farà servir necessàriament al model.

num_gbm = get_numeric(df_gbm)

# Eliminem la variable objectiu de la matriu X
X_gbm = num_gbm.drop(columns=["Has_claims_year"], errors="ignore")

# Inicialitzem PCA per, per exemple, 5 components
pca = PCA(n_components=5)
pca.fit(X_gbm)

# Guardem la proporció de variància explicada per cada component
pca_variance = pd.DataFrame({
    "Component": np.arange(1, 6),
    "Explained_variance": pca.explained_variance_ratio_
})
pca_variance.to_csv("data/processed/pca_variance_gbm.csv", index=False)

print("PCA exploratori completat i variàncies guardades a pca_variance_gbm.csv.")

# ------------------------------------------------------------
# 5) Importància de variables amb Gradient Boosting (GBM)
# ------------------------------------------------------------
# Fem servir un GradientBoostingClassifier bàsic per obtenir
# importàncies de variables sobre el dataset GBM (freqüència).
# Això proporciona un criteri addicional de selecció de variables.

gb = GradientBoostingClassifier()  # Model per defecte, només diagnòstic

X = X_gbm                          # predictors numèrics (sense Has_claims_year)
y = df_gbm["Has_claims_year"]      # target de freqüència

gb.fit(X, y)

# Extraiem importàncies i les ordenem de més a menys
importances = pd.DataFrame({
    "Variable": X.columns,
    "Importance": gb.feature_importances_
}).sort_values("Importance", ascending=False)

importances.to_csv("data/processed/gbm_importances.csv", index=False)
print("Importàncies GBM calculades i desades a gbm_importances.csv.")

# ------------------------------------------------------------
# 6) Selecció final (regla simple per GLM)
# ------------------------------------------------------------
# Combinar informació de:
#   - VIF > 10   → multicol·linearitat alta
#   - Correlació > 0.85 → parelles molt correlacionades
#
# Creem una llista de variables candidates a eliminar en el model GLM.

vars_drop = list(
    vif_glm[vif_glm["VIF"] > 10]["Variable"].unique()
) + list(
    high_corr_pairs[high_corr_pairs["Correlation"] > 0.85]["Var2"].unique()
)

# Eliminar duplicats
vars_drop = list(set(vars_drop))

# Guardar llista de variables candidates a exclusió
pd.Series(vars_drop).to_csv("data/processed/vars_to_drop_glm.csv", index=False)

print("Variables candidates a eliminació en GLM (VIF>10 o corr>0.85):")
print(vars_drop)

print("\nArxius exportats a data/processed/:")
print(" - vif_glm.csv                 (VIF per variable)")
print(" - high_corr_pairs_glm.csv    (parelles molt correlacionades)")
print(" - pca_variance_gbm.csv       (variància explicada per PCA en GBM)")
print(" - gbm_importances.csv        (importància de variables en GBM)")
print(" - vars_to_drop_glm.csv       (llista de variables a considerar eliminar)")
print("\n4.3.4.5 complet (reducció de dimensionalitat i anàlisi multivariant).")


Carregats:
GLM: (105555, 59)
GBM: (105555, 47)
SEV: (19646, 38)
VIF calculat per GLM i guardat a data/processed/vif_glm.csv.
Parelles amb correlació alta identificades i desades a high_corr_pairs_glm.csv.
PCA exploratori completat i variàncies guardades a pca_variance_gbm.csv.
Importàncies GBM calculades i desades a gbm_importances.csv.
Variables candidates a eliminació en GLM (VIF>10 o corr>0.85):
['Type_risk_ord_x_Area_0', 'Premium', 'Type_risk_ord_x_Area_1', 'Claims_to_premium_ratio', 'Driver_age_bin', 'Second_driver_1', 'Has_lapse_1', 'Power_cap', 'Type_risk_2', 'Power_log', 'Value_vehicle_cap_scaled', 'Has_claims_history_1', 'Has_lapse_0', 'Value_vehicle_bin', 'Type_fuel_D', 'Type_fuel_P', 'Value_vehicle_log', 'Type_risk_1', 'Driver_age_x_Power', 'Power_cap_scaled', 'Premium_cap_scaled', 'Power_bin', 'Has_claims_history_0', 'Driver_age', 'Type_risk_ord', 'Value_vehicle', 'Type_risk_4', 'Distribution_channel_1', 'Area_1', 'Type_fuel_Unknown', 'Second_driver_1_x_Power', 'Area_0', 'V

#### <b>4.3.4.6 Generació del dataset final per models</b>

In [14]:
# ============================================================
# 4.3.4.6 GENERACIÓ DEL DATASET FINAL PER A MODELS
# ============================================================
# Objectiu:
# - Carregar els datasets enriquits després de l'enginyeria de variables:
#   * Model de freqüència GLM/GAM (df_freq_fe_glm.csv)
#   * Model de freqüència GBM (df_freq_fe_gbm.csv)
#   * Model de severitat (df_sev_fe.csv)
# - Fer efectives les decisions de 4.3.4.5 (vars_to_drop_glm.csv)
#   sobre el dataset GLM (reducció de dimensionalitat i VIF),
#   PERÒ protegint explícitament les variables de segmentació actuarial.
# - Definir i verificar els camps clau (ID, Policy_year, set_type, targets).
# - Generar i desar datasets finals complets per:
#   * Model de freqüència (Has_claims_year) – GLM/GAM i Gradient Boosting
#   * Model de severitat (Cost_claims_year)
#   * Model de rendibilitat (Claims_to_premium_ratio) en un dataset separat
#     (ratio_full.csv), sense que aquesta ràtio entri com a predictor en freq/sev.
# - Deixar els datasets preparats per aplicar la divisió train/test
#   en un punt posterior (4.3.6).
# ============================================================

import os
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

# ------------------------------------------------------------
# 0) Càrrega datasets enriquits
# ------------------------------------------------------------
freq_glm_path = "data/processed/df_freq_fe_glm.csv"
freq_gbm_path = "data/processed/df_freq_fe_gbm.csv"
sev_path      = "data/processed/df_sev_fe.csv"

df_freq_glm = pd.read_csv(freq_glm_path)
df_freq_gbm = pd.read_csv(freq_gbm_path)
df_sev      = pd.read_csv(sev_path)

print("Carregats datasets enriquits:")
print("  Freq GLM:", df_freq_glm.shape)
print("  Freq GBM:", df_freq_gbm.shape)
print("  Severitat:", df_sev.shape)

ratio_path = "data/processed/df_ratio_num_transformed.csv"
if os.path.exists(ratio_path):
    df_ratio = pd.read_csv(ratio_path)
    print("  Ràtio econòmica:", df_ratio.shape)
else:
    df_ratio = None
    print("  No s'ha trobat df_ratio_num_transformed.csv. "
          "Es descarta dataset de ràtio en aquesta fase.")

# ------------------------------------------------------------
# 1) Camps clau i targets
# ------------------------------------------------------------
id_col   = "ID"
year_col = "Policy_year"
set_col  = "set_type"

target_freq  = "Has_claims_year"
target_sev   = "Cost_claims_year"
target_ratio = "Claims_to_premium_ratio"

# ------------------------------------------------------------
# 2) Aplicar decisions de 4.3.4.5 (reducció GLM)
#    protegint variables de segmentació actuarial
# ------------------------------------------------------------
vars_drop_path = "data/processed/vars_to_drop_glm.csv"

if os.path.exists(vars_drop_path):
    vars_drop_series = pd.read_csv(vars_drop_path, header=None)[0]

    vars_drop_raw = [
        v for v in vars_drop_series.tolist()
        if isinstance(v, str) and v.strip() != "" and v != "0"
    ]

    vars_drop_in_glm = [v for v in vars_drop_raw if v in df_freq_glm.columns]

    print("\n[Selecció GLM] Variables candidates a eliminació segons 4.3.4.5:")
    print(f"  Total a vars_to_drop_glm.csv : {len(vars_drop_raw)}")
    print(f"  Presentes a df_freq_glm      : {len(vars_drop_in_glm)}")

    # --- ⬇⬇⬇ CANVI IMPORTANT AQUI ⬇⬇⬇ ---
    # Protegim:
    #   - metadades i targets
    #   - variables de segmentació actuarial (dummies de risc, àrea, combustible, canal, lapse, històric, segon conductor)
    protected_cols = {
        id_col, year_col, set_col,
        target_freq, target_sev
    }

    # Prefixos de dummies de segmentació a protegir
    segmentation_prefixes = [
        "Type_risk_",
        "Area_",
        "Type_fuel_",
        "Distribution_channel_",
        "Second_driver_",
        "Has_lapse_",
        "Has_claims_history_",
    ]

    # Afegim totes les columnes de df_freq_glm que comencen per aquests prefixos
    for col in df_freq_glm.columns:
        if any(col.startswith(pref) for pref in segmentation_prefixes):
            protected_cols.add(col)

    # A partir d'aquí, només podem eliminar variables que:
    #   - surtin a vars_to_drop_in_glm
    #   - i NO siguin protegides
    vars_drop_final = [v for v in vars_drop_in_glm if v not in protected_cols]
    # --- ⬆⬆⬆ FI CANVI IMPORTANT ⬆⬆⬆ ---

    before_cols = df_freq_glm.shape[1]
    df_freq_glm = df_freq_glm.drop(columns=vars_drop_final, errors="ignore")
    after_cols = df_freq_glm.shape[1]

    print("\n[Selecció GLM] Resum de reducció de dimensionalitat:")
    print(f"  Columnes abans (df_freq_glm): {before_cols}")
    print(f"  Columnes després            : {after_cols}")
    print(f"  Nº de variables eliminades  : {len(vars_drop_final)}")
    if vars_drop_final:
        print("  Llista de variables eliminades en df_freq_glm:")
        print("   - " + "\n   - ".join(vars_drop_final))
    else:
        print("  No s'ha eliminat cap variable (totes protegides o no existents).")
else:
    print("\nAVÍS: No s'ha trobat 'vars_to_drop_glm.csv'.")
    print("   No s'aplica cap exclusió de variables al dataset GLM.")
    vars_drop_final = []

# ------------------------------------------------------------
# 2.bis) Assegurar que la ràtio econòmica NO entra al model GLM
# ------------------------------------------------------------
if target_ratio in df_freq_glm.columns:
    print(f"\n[Freq GLM] Eliminant {target_ratio} del dataset GLM "
          "(només s'utilitzarà com a target del model de ràtio).")
    df_freq_glm = df_freq_glm.drop(columns=[target_ratio])

# ------------------------------------------------------------
# 3) Comprovacions de metadades i targets
# ------------------------------------------------------------
for name, df in [
    ("Freq GLM", df_freq_glm),
    ("Freq GBM", df_freq_gbm),
    ("Severitat", df_sev),
]:
    missing = [c for c in [id_col, year_col, set_col] if c not in df.columns]
    if missing:
        raise ValueError(f"Falta/n columna/es {missing} al dataset {name}.")
print("\nMetadades (ID, Policy_year, set_type) presents a tots els datasets principals.")

for name, df, t in [
    ("Freq GLM", df_freq_glm, target_freq),
    ("Freq GBM", df_freq_gbm, target_freq),
    ("Severitat", df_sev, target_sev),
]:
    if t not in df.columns:
        print(f"Avís: el target {t} no es troba al dataset {name}.")

# ------------------------------------------------------------
# 4) Resum temporal (sense split encara)
# ------------------------------------------------------------
print("\nDistribució per set_type (Freq GLM):")
print(df_freq_glm[set_col].value_counts())

print("\nDistribució per set_type (Severitat):")
print(df_sev[set_col].value_counts())

# ------------------------------------------------------------
# 5) Desar datasets finals
# ------------------------------------------------------------
os.makedirs("data/model", exist_ok=True)

freq_glm_full_path = "data/model/freq_glm_full.csv"
freq_gbm_full_path = "data/model/freq_gbm_full.csv"
sev_full_path      = "data/model/sev_full.csv"

df_freq_glm.to_csv(freq_glm_full_path, index=False)
df_freq_gbm.to_csv(freq_gbm_full_path, index=False)
df_sev.to_csv(sev_full_path, index=False)

print("\nDatasets complets desats a:")
print(" -", freq_glm_full_path, "(freqüència GLM/GAM, amb reducció aplicada i sense Claims_to_premium_ratio)")
print(" -", freq_gbm_full_path, "(freqüència GBM)")
print(" -", sev_full_path,      "(severitat)")

if df_ratio is not None:
    ratio_full_path = "data/model/ratio_full.csv"
    df_ratio.to_csv(ratio_full_path, index=False)
    print(" -", ratio_full_path, "(ràtio econòmica, amb Claims_to_premium_ratio com a target)")

print("\n4.3.4.6 complet - Datasets finals consolidats generats.")
print(" Les decisions de 4.3.4.5 (VIF/correlacions) s'han aplicat al dataset GLM,")
print(" però preservant explícitament les variables de segmentació actuarial clau.")
print(" Claims_to_premium_ratio s'utilitza només en el model de ràtio econòmica,")
print(" i no entra com a predictor en els models de freqüència ni severitat.")
print(" La divisió train/test (validació temporal) es realitzarà al punt 4.3.6.")


Carregats datasets enriquits:
  Freq GLM: (105555, 59)
  Freq GBM: (105555, 47)
  Severitat: (19646, 38)
  Ràtio econòmica: (105555, 26)

[Selecció GLM] Variables candidates a eliminació segons 4.3.4.5:
  Total a vars_to_drop_glm.csv : 38
  Presentes a df_freq_glm      : 38

[Selecció GLM] Resum de reducció de dimensionalitat:
  Columnes abans (df_freq_glm): 59
  Columnes després            : 41
  Nº de variables eliminades  : 18
  Llista de variables eliminades en df_freq_glm:
   - Premium
   - Claims_to_premium_ratio
   - Driver_age_bin
   - Power_cap
   - Power_log
   - Value_vehicle_cap_scaled
   - Value_vehicle_bin
   - Value_vehicle_log
   - Driver_age_x_Power
   - Power_cap_scaled
   - Premium_cap_scaled
   - Power_bin
   - Driver_age
   - Value_vehicle
   - Value_vehicle_cap
   - Cost_claims_year_log
   - Premium_cap
   - Vehicle_age_bin

Metadades (ID, Policy_year, set_type) presents a tots els datasets principals.

Distribució per set_type (Freq GLM):
set_type
train    69740
